In [1]:
from utils import clean_dataframe, load_dataframes, build_freq_words_list, tf_idf_prediction

In [2]:
df = load_dataframes()

In [3]:
df = clean_dataframe(df)

In [7]:
bio = df[df.site == 'biology']

In [15]:
bio.shape

(13196, 6)

In [16]:
unique_true_tags = set()
for words in bio.tags.values:
    words = words.split()
    for word in words:
        unique_true_tags.add(word)

In [17]:
len(unique_true_tags)

678

In [8]:
freq_list, words_list = build_freq_words_list(bio)

0.0%
37.89026977872082%
75.78053955744164%


In [10]:
predictions = tf_idf_prediction(freq_list, words_list, threshold=0.3)

In [12]:
unique_words = set()

In [103]:
from collections import Counter

c = Counter()

## count the word frequencies

In [104]:
# get unique words and count frequencies
for words in predictions:
    for word in words:
        c[word] += 1
        unique_words.add(word)

In [18]:
N_TARGET_TAGS = 700

## transform the words into vectors with gensim

In [22]:
from gensim.models import Word2Vec

In [32]:
import numpy as np

model can be found at https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [27]:
model = Word2Vec.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [83]:
words_word2vec = []
success_words = []
not_found_words = set()
for word in unique_words:
    try:
        words_word2vec.append(model[word])
        success_words.append(word)
    except KeyError:
        not_found_words.add(word)

In [84]:
for i, word in enumerate(not_found_words):
    if i < 10:
        print(word)

adrenoreceptors
wpsl
spermatocyte
glycosylases
inse
pitx
suvu
mirna
antiperistalsis
zcmfo


#### Some unknown words actually exist!

### cluster with k-means

In [85]:
from sklearn.cluster import KMeans

In [86]:
import pandas as pd

In [87]:
X = np.array(words_word2vec)

In [122]:
N_CLUSTERS = 700

In [88]:
km = KMeans(n_clusters=N_CLUSTERS)

In [89]:
%%time
km.fit(X)

CPU times: user 1min 57s, sys: 4.9 s, total: 2min 2s
Wall time: 1min 39s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=700, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [90]:
clusters = km.predict(X)

In [96]:
success_words = np.array(success_words)

In [115]:
# for each cluster, we find the best word
# we create two kinds of dict
# word => cluster
# cluster => best representant

In [116]:
word_to_cluster = {}

In [117]:
cluster_to_representative = {}

In [123]:
%%time
for i in range(N_CLUSTERS):
    # words in the i-th cluster
    words_in_cluster = success_words[clusters == i]

    for word in word_to_cluster:
        word_to_cluster[word] = i

    # most used frequency of the word
    cluster_to_representative[i] = sorted(words_in_cluster, key=lambda x: c[x], reverse=True)[0]

CPU times: user 37.5 ms, sys: 10.3 ms, total: 47.8 ms
Wall time: 109 ms


## how do the reprensentative do?

In [127]:
representatives = set([v for k, v in cluster_to_representative.items()])

In [130]:
len(representatives.intersection(unique_true_tags))

91